In [ ]:
import pandas as pd

quick = True

if quick:
    start_year, end_year, chk, sampled, filename = [2005, 2021, False, True, 'df-light.pkl']
else:
    start_year, end_year, chk, sampled, filename = [2005, 2021, True, False, 'df-full.pkl']

from sklearn.model_selection import train_test_split

df = pd.read_pickle(f'./{filename}')
data = df.iloc[:, 1:]
target = df['grav']

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=222)

In [ ]:
import time
from my_libs.encoder_custom import EncoderCustom

start_time = time.time()

# cols_target_encoded = ['dep', 'age']
cols_target_encoded = []
cols_onehot_encoded = X_train.columns.drop(cols_target_encoded)

encoder = EncoderCustom(cols_target_encoded=cols_target_encoded, cols_onehot_encoded=cols_onehot_encoded)
X_train_rs, y_train_rs = encoder.transform(X_train, y_train, 'Train')
X_test_rs,  y_test_rs  = encoder.transform(X_test,  y_test,  'Test')

print("--- Features encoding performed in %s seconds ---" % (time.time() - start_time))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report

start_time = time.time()

params = {'classifier': 'RandomForestClassifier', 'n_estimators': 26, 'criterion': 'entropy', 'max_depth': 27, 'min_samples_split': 3}

if params['classifier'] == 'RandomForestClassifier' :
    model_simple = RandomForestClassifier(n_estimators=params['n_estimators'],
                                   criterion=params['criterion'],
                                   max_depth=params['max_depth'],
                                   min_samples_split=params['min_samples_split'])

model_simple.fit(X_train_rs, y_train_rs)

y_pred = model_simple.predict(X_test_rs)

print("--- Model fit and trained in %s seconds ---" % (time.time() - start_time))

print(pd.crosstab(y_test_rs, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))

# print("Classification report imbalanced :")
# print(classification_report_imbalanced(y_test_rs, y_pred))

print("Classification report :")
print(classification_report(y_test_rs, y_pred))